In [1]:
import os

In [ ]:
import xmltodict
import os

full_names = []
annot_folder = "/wynton/group/andrews/data/shhs/nsrr/polysomnography/annotations-events-profusion/shhs1/"
for file in os.listdir(annot_folder)[:300]:
    if file.endswith(".xml"):
        annot_path = os.path.join(annot_folder, file)
        with open(annot_path, encoding='utf-8') as f:
            info_dict = xmltodict.parse(f.read())
        events = info_dict['CMPStudyConfig']['ScoredEvents']['ScoredEvent']
        for event in events:
            name = event['Name']
            if name not in full_names:
                full_names.append(name)
print(full_names)

['SpO2 artifact', 'Arousal ()', 'Hypopnea', 'SpO2 desaturation', 'Obstructive Apnea', 'Central Apnea', 'Arousal (STANDARD)', 'Mixed Apnea', 'Arousal (ASDA)', 'Unsure', 'Arousal (CHESHIRE)', 'Arousal (Standard)']


In [ ]:
import pandas as pd
path = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/extracted_features/hsp_mgb/run_20251218/hsp_mgb_all.csv"
df = pd.read_csv(path)
print("DOMINANT")
df[["AHI_NREM","AHI_REM"]]

In [ ]:
import pandas as pd
path = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/extracted_features/hsp_mgb/run_20251218/hsp_mgb_all.csv"
df = pd.read_csv(path)
print("ONSET")
df[["AHI_NREM","AHI_REM"]]

In [ ]:
import pandas as pd
path = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/extracted_features/hsp_mgb/run_20251218/hsp_mgb_all.csv"
df = pd.read_csv(path)
print("OFFSET")
df[["AHI_NREM","AHI_REM"]]

In [ ]:
path = "/wynton/group/andrews/data/HSP/PSG/bids/MGB"

import os
import pandas as pd
for sub in os.listdir(path)[:200]:
    sub_path = os.path.join(path,sub)
    for ses in os.listdir(sub_path):
        ses_path = os.path.join(sub_path, ses, "eeg")
        if os.path.exists(ses_path):
            for file in os.listdir(ses_path):
                if "channels" in file:
                    df = pd.read_csv(os.path.join(ses_path, file), sep = "\t")
                    names = df["name"].to_list()
                    if "NPT" in names:
                        print(file)

In [ ]:
import os
import pandas as pd
import h5py

psg_ids = [
    ("S0001114191434", 1),
    ("S0001116639282", 1),
    ("S0001121913093", 1),
    ("S0001122312754", 1),
]

base_path = "/wynton/group/andrews/data/HSP/PSG/bids/MGB/"

# Set pandas display options to show all rows/columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Loop over subjects
for sub_id, session in psg_ids:
    eeg_path = os.path.join(base_path, f"sub-{sub_id}", f"ses-{session}", "eeg")
    
    if not os.path.exists(eeg_path):
        print(f"[WARNING] Path not found: {eeg_path}")
        continue

    # Look for TSV files (usually *_channels.tsv in BIDS EEG)
    tsv_files = [f for f in os.listdir(eeg_path) if f.endswith("channels.tsv")]
    if not tsv_files:
        print(f"[WARNING] No channels TSV found in {eeg_path}")
        continue

    # Open the first TSV found
    tsv_file_path = os.path.join(eeg_path, tsv_files[0])
    df_channels = pd.read_csv(tsv_file_path, sep='\t')
    
    print(f"\nSubject {sub_id}, session {session}: {tsv_file_path}")
    print(df_channels['name'])  # Print the full dataframe

In [ ]:
import os

path_log = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/log/vb_shhs1.o1611326"
path_features = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/extracted_features/shhs_ses-1/run_20251216/"

selected_subjects = []

with open(path_log, "r") as f:
    lines = f.readlines()
    for line in lines: 
        if "[ERROR] MATLAB" in line:
            psg_id = line.split('.mat')[0].split('sub-')[-1]
            sub_id = psg_id.split('_')[0]
            session = psg_id.split('ses-')[-1]
            selected_subjects.append((sub_id, int(session)))
            # delete files
            file = os.path.join(path_features,
                    f"shhs_ses-1_sub-{sub_id}_extracted_features.csv")
            if os.path.exists(file):
                #print(file)
                os.remove(file)
            # delete file wide
            file_wide = os.path.join(path_features,
                    f"shhs_ses-1_sub-{sub_id}_extracted_features_wide.csv")
            if os.path.exists(file_wide):
                #print(file_wide)
                os.remove(file_wide)
# Remove duplicates
selected_subjects = list(set(selected_subjects))

print(selected_subjects)
print(len(selected_subjects), "subjects/sessions with missing features.")

In [ ]:
# Get the list of files to keep 
master = pd.read_csv("/wynton/group/andrews/data/PSG_Pipeline_Outputs/mastersheets/shhs_ses-1_mastersheet.csv")


for sub in 
master["h5_path"] = master["h5_path"].str.split('/').str[-1]
keep_files = set(master["h5_path"].to_list())

# Go though all the files computed
path_h5 = "/wynton/group/andrews/data/PSG_Pipeline_Outputs/h5_data/hsp_mgb/"
for file in os.listdir(path_h5):
    if file not in keep_files:
        print(f"Deleting: {file}")
        #os.remove(os.path.join(path_h5, file)) 

In [ ]:
path_log = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/log/h5_shhs1.o1467406"
# path_log = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/log/burdens_shhs2.o1467690"
selected_subjects = []

with open(path_log, "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines[:-1]):  # avoid going past last line
    if "No RESP" in line and "No RESP" in lines[i+1]:
        psg_id = line.split('sub-')[-1].strip()
        sub_id = psg_id.split('_')[0]
        session = psg_id.split('ses-')[-1]
        selected_subjects.append((sub_id, int(session)))
        print(line)
        print(lines[i+1])

# Remove duplicates
selected_subjects = list(set(selected_subjects))

print(selected_subjects)
print(len(selected_subjects), "subjects/sessions with missing features.")

In [ ]:
path_log = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/log/burdens_shhs1.o1467689"
# path_log = "/wynton/home/leng/alice-albrecht/projects/PSG_Pipeline/log/burdens_shhs2.o1467690"
selected_subjects = []
with open(path_log, "r") as f:
    for line in f:
        if "subprocess.CalledProcessError:" in line:
            psg_id = line.split('.mat')[0].split('sub-')[-1]
            sub_id = psg_id.split('_')[0]
            session = psg_id.split('ses-')[-1]
            selected_subjects.append((sub_id, int(session)))
        if "Feature 'vb' was not computed" in line:
            sub_id = line.split(':')[0].split('Sub ')[-1].strip()
            session = 1
            selected_subjects.append((sub_id, int(session)))

print(len(selected_subjects))

In [ ]:
import xmltodict
from collections import Counter

annot_path = "/wynton/group/andrews/data/MrOS/mros-sof_mjhe/vs/EDF/bi/bi0002.edf.XML"
with open(annot_path, encoding='utf-8') as f:
        info_dict = xmltodict.parse(f.read())

events = info_dict['CMPStudyConfig']['ScoredEvents']['ScoredEvent']
sleep_stages = info_dict['CMPStudyConfig']['SleepStages']['SleepStage']
names = [event['Name'] for event in events if 'Name' in event]
Counter(names)

In [ ]:
import os
import pandas as pd 
pd.set_option('display.max_rows', None)

df = pd.read_csv("/Users/alicealbrecht/Desktop/S0001_psg_metadata_2025-09-05.csv")
print(df.shape)
df = df.rename(columns={
    "BDSPPatientID": "sub_id",
    "SessionID": "session"
})
df['sub_id'] = 'S0001' + df['sub_id'].astype(str)
df['session'] = df['session'].astype(int)
df = df[
    df['StudyType'].str.lower().str.contains("diagnostic|dignostic", na=False) &
    (~df['StudyType'].str.lower().str.contains("oxygen", na=False))
]
print(df.shape)
df['StudyType'].value_counts()

In [ ]:
master = pd.read_csv("/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet_before_diagnostic.csv")
print(master.shape)
master['sub_id'] = master['sub_id'].astype(str)
master['session'] = master['session'].astype(int)
master = master.merge(
    df[['sub_id', 'session', 'StudyType']],
    on=['sub_id', 'session'],
    how='left'
)
master = master[
    master['StudyType'].str.lower().str.contains("diagnostic|dignostic", na=False) &
    (~master['StudyType'].str.lower().str.contains("oxygen", na=False))
]
print(master.shape)
master['StudyType'].value_counts()

In [ ]:
master = master.drop(columns= ['StudyType'])
master.shape

In [ ]:
output_path = "/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet.csv"
master.to_csv(output_path, index=False)

In [ ]:
master = master[~master['annot_path'].isna()]
print(master.shape)
# Look at all possible annotaiotn file found
possible_annot = master['annot_path'].str.extract(r'ses-\d+_(.*)')[0]
possible_annot.unique()

In [ ]:
import pandas as pd
master = pd.read_csv("/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet.csv")
master['annot_options'] = master['annot_path'].str.extract(r'ses-\d+_(.*)')[0]
master['annot_options'].value_counts()

In [ ]:
master.shape